### Preprocessing

In [1]:
# Import packages
import os
os.chdir("../src/preprocessing") # Change current working directory to load preprocessing functions
from create_embedding import preprocess_folder, apply_embedding_model_to_svgs, combine_embeddings, encode_pkl, encode_svg, decode_z
from deepsvg.svglib.svg import SVG
from deepsvg import utils
os.chdir("../..")

In [2]:
from src.data.insert_ids import insert_ids_in_folder
from src.data.get_svg_meta_data import get_svg_meta_data
from src.data.generate_random_animations import random_animation_vector
from src.preprocessing.sort_paths import sort_by_relevance
from src.preprocessing.decompose_logo import decompose_logos_in_folder
from src.features.get_style_attributes import *
#from src.features.get_style_attributes_folder import get_local_style_attributes, get_global_style_attributes, combine_style_attributes
from src.models_output.insert_animation import create_animated_svg
from src.models_output.transform_animation_predictor_output import transform_animation_predictor_output
import pandas as pd
import numpy as np
import importlib, glob, pickle, torch, logging
from datetime import datetime
from concurrent import futures
from tqdm import tqdm

#### 1 Insert IDs in SVGs
An ID called animation_id is inserted for every path in a SVG.

In [3]:
insert_ids_in_folder("data/svgs_expanded")

#### 2 Decompose SVGs
Decompose SVGs by saving every path in a seperate SVG file. Decomposed SVGs are save to data/decomposed_svgs.

In [4]:
decompose_logos_in_folder("data/svgs")

#### 3 Simplify SVGs
Creates simplified SVGs and its tensors on SVG and path level

In [7]:
preprocess_folder(data_folder="data/svgs")

100%|████████████████████████████████████████████████████████████████████████████████| 192/192 [00:54<00:00,  3.52it/s]


In [8]:
preprocess_folder(data_folder="data/decomposed_svgs")

100%|██████████████████████████████████████████████████████████████████████████████| 5109/5109 [02:42<00:00, 31.53it/s]


#### 4 Get Path Embeddings
Apply hierarchical generative network to produce latent vector z

In [4]:
model = "models/20210301_1813_model_batch60_epoch30_decomposed_svgs.pth.tar"

In [5]:
#svgs_embedding = apply_embedding_model_to_svgs(model_path=model, data_folder="data/svgs", save=False)

In [6]:
decomposed_svgs_embedding = apply_embedding_model_to_svgs(model_path=model, data_folder="data/decomposed_svgs", save=True)

100%|██████████████████████████████████████████████████████████████████████████████| 5109/5109 [09:32<00:00,  8.93it/s]


In [ ]:
#combined_embedding = combine_embeddings(df_svg_embedding=svgs_embedding, df_decomposed_svg_embedding=decomposed_svgs_embedding, model_path=model, save=False)

In [9]:
decomposed_svgs_embedding

,filename,animation_id,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,L_7sins_01_10,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L_7sins_01_11,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,L_7sins_01_1,1,1.058049,-0.221425,1.557320,-0.180708,-1.182121,0.801264,0.992946,1.265655,...,-2.367993,-1.322433,-1.019614,-0.560317,0.006592,-0.656671,-0.971138,-1.833762,1.170980,0.299563
3,L_7sins_01_0,0,0.573274,-0.169635,1.547139,-0.351954,-0.938741,0.352232,0.770248,1.144378,...,-2.630614,-1.088127,-0.493820,-1.005420,0.040128,-0.295772,-0.682393,-1.680727,1.112269,-0.015597
4,L_7sins_01_14,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5076,L_Weidenbachhof_9_2_4,4,0.789945,0.071214,1.702776,-0.441861,-1.201321,0.604801,1.136409,1.076383,...,-2.226045,-1.005836,-0.615871,-0.403113,-0.045758,-0.361017,-0.315227,-1.989081,1.048624,0.578470
5077,L_Weidenbachhof_9_2_6,6,0.784052,0.049785,1.310274,-0.556812,-1.248168,1.086187,0.729777,1.235167,...,-1.918421,-1.250743,-0.621715,-0.353701,0.304269,-0.660002,-0.422328,-2.391057,1.110940,0.514435
5078,L_Weidenbachhof_9_2_7,7,1.056329,0.010147,1.486216,-0.541472,-1.205664,0.974453,0.917410,1.520458,...,-2.000242,-1.159243,-0.693862,-0.452490,0.158430,-0.778412,-0.669429,-2.013637,1.229304,0.485261
5079,L_Weidenbachhof_9_2_8,8,1.327888,-0.738074,1.614826,-0.129609,-1.069742,1.113564,1.060841,1.533815,...,-2.406755,-1.027968,-0.806419,-0.673152,-0.188934,-0.878584,-0.514161,-1.902013,1.157261,0.620449


In [6]:
#encode_svg("data/decomposed_svgs/L_7sins_01_0.svg", model_path=model, data_folder="data/decomposed_svgs")

#### 5 Get Style Attributes
Get local as well as global style attributes of SVGs.

In [7]:
df_local = get_local_style_attributes("data/svgs")
df_global = get_global_style_attributes("data/svgs")

In [8]:
df_all_style_attributes = combine_style_attributes(df_global, df_local)
df_all_style_attributes

,file,animation_id,class_,fill,stroke,stroke_width,opacity,stroke_opacity
0,L_7sins_01.svg,0,st0,#FABB00,#000000,0,1.0,1.0
1,L_7sins_01.svg,1,st1,#C5007B,#000000,0,1.0,1.0
2,L_7sins_01.svg,2,st2,#EC7404,#000000,0,1.0,1.0
3,L_7sins_01.svg,3,st1,#C5007B,#000000,0,1.0,1.0
4,L_7sins_01.svg,4,st2,#EC7404,#000000,0,1.0,1.0
...,...,...,...,...,...,...,...,...
5095,L_Weidenbachhof_9_2.svg,22,st3,#FFFFFF,#000000,0,1.0,1.0
5096,L_Weidenbachhof_9_2.svg,23,st3,#FFFFFF,#000000,0,1.0,1.0
5097,L_Weidenbachhof_9_2.svg,24,st3,#FFFFFF,#000000,0,1.0,1.0
5098,L_Weidenbachhof_9_2.svg,25,st3,#FFFFFF,#000000,0,1.0,1.0


In [ ]:
#df_all_style_attributes.to_csv("data/style_attributes.csv")

In [ ]:
#output = open("data/style_attributes.pkl", 'wb')
#pickle.dump(df_all_style_attributes, output)
#output.close()

#### 6 Insert Animations

In [5]:
file = "data/svgs/L_7sins_02.svg"
animation_id = sort_by_relevance("data/path_selection/L_7sins_02")

In [6]:
for s in range(200):
    model_output = random_animation_vector(8, seed=s)
    create_animated_svg(file, animation_id, model_output, str(s))